In [1]:
!wget https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release -O sqlite.tar.gz
!tar xzf sqlite.tar.gz
%cd sqlite/
!./configure
!make sqlite3.c
%cd /content
!npx degit coleifer/pysqlite3 -f
!cp sqlite/sqlite3.[ch] .
!python setup.py build_static build
!cp build/lib.linux-x86_64-3.6/pysqlite3/_sqlite3.cpython-36m-x86_64-linux-gnu.so \
    /usr/lib/python3.6/lib-dynload/_sqlite3.cpython-36m-x86_64-linux-gnu.so

--2020-12-03 06:07:13--  https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release
Resolving www.sqlite.org (www.sqlite.org)... 45.33.6.223, 2600:3c00::f03c:91ff:fe96:b959
Connecting to www.sqlite.org (www.sqlite.org)|45.33.6.223|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11101880 (11M) [application/x-compressed]
Saving to: ‘sqlite.tar.gz’

sqlite.tar.gz       100%[===================>]  10.59M  37.5MB/s    in 0.3s    

2020-12-03 06:07:14 (37.5 MB/s) - ‘sqlite.tar.gz’ saved [11101880/11101880]

/content/sqlite
checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
checking for gcc... gcc
checking whether the C compiler works... yes
checking for C compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether we are using the GNU C compiler... yes
checking whether gcc accepts -g... yes
chec

In [69]:
import pandas as pd
import numpy as np
import sqlite3
conn = sqlite3.connect('db')
cur = conn.cursor()
sqlite3.sqlite_version

'3.34.0'

In [70]:
t1 = pd.DataFrame( {'Код_менеджера' : [1,2,3,4,5,6,7,8], 
                    'Тип_точки': [1,1, 1,2,2,2,1,2], 
                    'Продажи_тонн': [100,150, np.nan, 250, 50, 300,90,120]}
                  )

In [8]:
# мелкие точки Мск
t2 = pd.DataFrame( {'Код_менеджера' : [1,2], 
                    'ФИО_менеджера': ['aa', 'bb']}
                  )

In [9]:
# мелкие точки спб
t3 = pd.DataFrame( {'Код_менеджера' : [3,7], 
                    'ФИО_менеджера': ['cc', 'dd']}
                  )

In [53]:
# крупные точки мск и спб
t4 = pd.DataFrame( {'Код_менеджера' : [4,5,6,8,1,7], 
                    'ФИО_менеджера': ['ss', 'ff', 'gg', 'pp', 'aa', 'dd'],
                    'Город' : ['Мск', 'Мск', 'Спб', 'Мск', 'Мск', 'Спб']}
                  )

In [71]:
t1.to_sql('t1', conn, index=False, if_exists='replace')
t2.to_sql('t2', conn, index=False, if_exists='replace')
t3.to_sql('t3', conn, index=False, if_exists='replace')
t4.to_sql('t4', conn, index=False, if_exists='replace')

In [13]:
sql = ''' select * from t1'''

In [14]:
pd.read_sql(sql,conn)

,Код_менеджера,Тип_точки,Продажи_тонн
0,1,1,100
1,2,1,150
2,3,1,200
3,4,2,250
4,5,2,50
5,6,2,300
6,7,1,90
7,8,2,120


In [15]:
# фукнция для чтения и вывода запроса
def select(sql):
  return pd.read_sql(sql,conn)

In [16]:
select(sql)

,Код_менеджера,Тип_точки,Продажи_тонн
0,1,1,100
1,2,1,150
2,3,1,200
3,4,2,250
4,5,2,50
5,6,2,300
6,7,1,90
7,8,2,120


Существуют менеджеры, которые могут обслуживать и мелкие, и крупные торговые точки одного города.

То есть часть менеджеров во 2 и 4 таблице могут пересекаться, а также часть менеджеров в 3 и 4 таблице могут пересекаться.



1) Продажи в разрезе менеджеров (ФИО), которые одновременно обслуживают и крупные, и мелкие точки Москвы

In [41]:
sql = '''
select t.ФИО_менеджера, sum(t1.Продажи_тонн) from (
select * from t2 where exists ( select * from t4 where t2.Код_менеджера=t4.Код_менеджера)
union
select t4.Код_менеджера, t4.ФИО_менеджера from  t4 where exists ( select * from  t2 where t4.Код_менеджера=t2.Код_менеджера )
) t
left join t1 on t1.Код_менеджера=t.Код_менеджера
order by 1
'''

In [42]:
select(sql)

,ФИО_менеджера,sum(t1.Продажи_тонн)
0,aa,100


2) Продажи в разрезе менеджеров (ФИО), обслуживающих только крупные точки в Санкт-Петербурге

In [67]:
sql = '''
Select t.ФИО_менеджера, sum(t1.Продажи_тонн) from (
select t4.Код_менеджера, t4.ФИО_менеджера from  t4 where not exists 
( select * from  t3 where t4.Код_менеджера=t3.Код_менеджера )
and t4.Город == 'Спб'
) t
left join t1 on t1.Код_менеджера=t.Код_менеджера
order by 1
'''

In [68]:
select(sql)

,ФИО_менеджера,sum(t1.Продажи_тонн)
0,gg,300


3) Полный список менеджеров, работающих в Санкт-Петербурге. Напротив каждого менеджера указать суммарную величину его продаж, если продаж нет, то указываем 0

In [106]:
sql = '''

select a.ФИО_менеджера, sum(a.rev) from (
select t4.ФИО_менеджера, ifnull(t1.Продажи_тонн,0) as rev from t4

left join t1 on t1.Код_менеджера=t4.Код_менеджера
where t4.Город='Спб'
union
select t3.ФИО_менеджера, ifnull(t1.Продажи_тонн,0) as rev from t3
left join t1 on t1.Код_менеджера=t3.Код_менеджера
) a
group by a.ФИО_менеджера
'''

In [107]:
select(sql)

,ФИО_менеджера,sum(a.rev)
0,cc,0.0
1,dd,90.0
2,gg,300.0


4) Добавить к запросу п.3 условие, при котором будут выводиться продажи только по тем менеджерам, которые суммарно продали больше 100 тонн

In [110]:
sql = '''
select a.ФИО_менеджера, sum(a.rev) from (
select t4.ФИО_менеджера, ifnull(t1.Продажи_тонн,0) as rev from t4

left join t1 on t1.Код_менеджера=t4.Код_менеджера
where t4.Город='Спб'
union
select t3.ФИО_менеджера, ifnull(t1.Продажи_тонн,0) as rev from t3
left join t1 on t1.Код_менеджера=t3.Код_менеджера
) a
group by a.ФИО_менеджера
having sum(a.rev)>100
'''

In [111]:
select(sql)

,ФИО_менеджера,sum(a.rev)
0,gg,300.0


5) Удалить из таблицы 1 данные с Тип_точки "Мелкая"

In [ ]:
sql = '''
delete from t1
where t1.Тип_точки = 'Мелкая'
'''

6) Добавить в таблицу 1 данные с Тип_точки "Мелкая", которые будут равны данным с Тип_точки "Крупная"